In [16]:
#Important packages
import math
import numpy as np
from scipy.integrate import quad
import scipy.linalg as sla
from __future__ import division

In [17]:
# -*- coding: utf-8 -*-
"""
For information and usage see README, or http://pypi.python.org/pypi/numericalunits
"""
#Copyright (C) 2012-2017 Steven Byrnes
#
#Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#
#The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
#
#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

#From the numericalunits package: https://pypi.org/project/numericalunits/

#Set all variables to help introspection libraries
m = kg = s = C = K = 0.

g = 0.

cm = mm = um = nm = pm = fm = km = angstrom = lightyear = \
    astro_unit = pc = kpc = Mpc = Gpc = inch = foot = mile = thou = 0.
    
ms = us = ns = ps = fs = minute = hour = day = week = year = 0.
    
J = mJ = uJ = nJ = pJ = fJ = kJ = MJ = GJ = erg = eV = meV = keV = MeV = GeV = \
    TeV = btu = smallcal = kcal = Wh = kWh = 0.
    
def reset_units(seed=None):
    
    #Set all units to new self-consistent, floating-point values. 
    
    """
    reset_units() --> units are randomized. This is the suggested use. Run this 
    before your calculation, display the final answer, then re-run this, then re-display
    the final answer. If you get the same answers both times, then your calculations
    are almost guaranteed to be free of dimensional-analysis-violating errors.
    This method is run automatically the first time this module is imported.
    
    reset_unit('SI') --> Set units so that all values are given in standard SI
    units (meters-kilograms-seconds) by default. In this mode, there is no way to test
    for dimensional-analysis-violating errors.
    
    reset_units(x) --> If you pass any other argument x, it's used as the seed
    for the random number generator.
    
    """
    
    import random
    
    global m, kg, s, C, K
    
    if seed == 'SI':
        m = 1.
        kg = 1.
        s = 1.
        C = 1.
        K = 1.
    else: 
        prior_random_state = random.getstate()
        
        if seed is Non: 
            random.seed()
        else:
            random.seed(seed)
        
        m = 10. ** random.uniform(-1, 1) #meter
        kg = 10. ** random.uniform(-1, 1) #kilogram
        s = 10. ** random.uniform(-1, 1) #second
        C = 10. ** random.uniform(-1, 1) #coulombs
        K = 10. ** random.uniform(-1, 1) #kelvins
        
        #Leave the random generator like found, in case someone else is using it.
        random.setstate(prior_random_state)
    
    set_derived_units_and_constants()
    return

def set_derived_units_and_constants():
    
    """
    Assuming that the base units (m, kg, s, C, K) have already been set as 
    floating point values, this function sets all other units and constants to the
    appropriate, self-consistent values.
    
    """
    
    #Length
    global cm, mm, km, lightyear, astro_unit, pc, kpc, Mpc, Gpc, inch, foot, mile
        
    cm = 1e-2 * m
    mm = 1e-3 * m
    km = 1e3 * m
    lightyear = 9460730472580800. * m
    astro_unit = 149597870700. * m #astronomical unit
    pc = (648000./math.pi) * astro_unit #parsec
    kpc = 1e3 * pc
    Mpc = 1e6 * pc
    Gpc = 1e9 * pc
    inch = 2.54 * cm
    foot = 12. * inch
    mile = 5280. * foot
    
    #Mass
    global g
    
    g = 1e-3 * kg
    
    #Time
    global minute, hour, day, week, year
    
    minute = 60. * s
    hour = 60. * minute
    day = 24. * hour #solar day
    week = 7. * day
    year = 365.256363004 * day #sidereal year
    
    #Energy
    global J, erg, eV, keV, MeV, GeV, Tev
    
    J = (kg * m**2) / s**2
    erg = 1e-7 * J
    eV = 1.6021766208e-19 * J
    keV = 1e3 * eV
    Mev = 1e6 * eV
    GeV = 1e9 * eV
    TeV = 1e12 * eV

In [33]:
#Necessary constants
M1 = 1.75
M2 = 2.5

#compactness, prompt user to enter either 1.75 or 2.5 
comp175 = float(raw_input("What is the compactness (psi_1.75)?"))
comp25 = float(raw_input("What is the compactness (psi_2.5)?"))  

What is the compactness (psi_1.75)?0.7
What is the compactness (psi_2.5)?0.2


In [43]:
reset_units('SI')
set_derived_units_and_constants()

M_sun = 1.9891e30 * kg

#Density profile based on radius r
def densityProfile(r):
    rho_0 = 6e11 * (g / cm**3)
    r_star = 1e7 * cm
    return rho_0 / (1 + (r / r_star)**3)

#Enclosed mass 
def enclosedMass(r_upper, rs, n):
    MR = lambda r: ((4*math.pi)*(densityProfile(r))*(r**2)*((r / rs)**(3*n))) / M_sun
    result = quad(MR, 0, r_upper)
    return result[0]
    
def broydenMethod(x, y, f_eqns, J_eqns, tol = 10e-1, maxIterations = 50):
    steps_taken = 0
        
    f = f_eqns(x, y)
    J = J_eqns(x, y)
        
    while np.linalg.norm(f, 2) > tol and steps_taken < maxIterations:
            
        s = sla.solve(J, -1 * f)
            
        x = x + s[0]
        y = y + s[1]
        print(x,y)    
        new_f = f_eqns(x, y)
        z = new_f - f
            
        J = J + (np.outer((z - np.dot(J, s)), s)) / (np.dot(s, s))
            
        f = new_f
        print(f)
        print(np.linalg.norm(f, 2))
        print (tol)
        steps_taken += 1
            
    return steps_taken, x, y
        
tol = 10.0** -7
maxIterations = 50
x0 = 1e8
y0 = 0.01
 
def fs(rs, n):
    return np.array([enclosedMass(((1000 * km) * M1) / comp175, rs, n ) - M1, 
                     enclosedMass(((1000 * km) * M2) / comp25, rs, n ) - M2])  #Define the f function here
 
def Js(rs, n):
    dmdn = lambda r: ((densityProfile(r))*(r**2)*((r / rs)**(3*n)) * math.log(r)) * 12 * math.pi
    return np.array([[((-3*n)/rs) * enclosedMass(((1000 * km) * M1) / comp175, rs, n ), (quad(dmdn, 0, (1000 * km))[0] * M1) / comp175],
                     [((-3*n)/rs) * enclosedMass(((1000 * km) * M2) / comp25, rs, n ), (quad(dmdn, 0, (1000 * km))[0] * M2) / comp25]])

print(((1000 * km) * M1) / comp175)
print(np.array([enclosedMass(((1000 * km) * M1) / comp175, 1, 0 ) - M1, 
                     enclosedMass(((1000 * km) * M2) / comp25, 1, 0 ) - M2]))

n, x, y = broydenMethod(x0, y0, fs, Js, tol, maxIterations )
print "iterations: ", n
print "x and y: ", x, y

2500000.0
[10.45145432 15.80206083]
(2854411057.743871, 0.01)
[ 7.65110302 11.96135958]
14.199066883721029
1e-07
(23894386672.401367, 0.01)
[ 7.07054522 11.06830957]
13.13392882476568
1e-07
(278128160524.2738, 0.01)
[ 6.4443962  10.10512833]
11.9851516852459
1e-07
(2872215964967.594, 0.01)
[5.89007804 9.25244176]
10.968167473814908
1e-07
(30176992441870.38, 0.01)
[5.36957433 8.45177069]
10.013229060744866
1e-07
(308947220938018.5, 0.01)
[4.88968962 7.71358227]
9.132820816877945
1e-07
(3117208553235625.5, 0.01)
[4.44485765 7.02931415]
8.31673114914816
1e-07
(3.082789579837212e+16, 0.01)
[4.03330359 6.39623617]
7.561704505554015
1e-07
(2.986639479670147e+17, 0.01)
[3.65242596 5.81034658]
6.862968945093985
1e-07
(2.828083799058204e+18, 0.01)
[3.3000949 5.2683691]
6.216617995381867
1e-07
(2.612170413118454e+19, 0.01)
[2.97426512 4.76715754]
5.618900608564854
1e-07
(2.3478808184794966e+20, 0.01)
[2.67307217 4.30384387]
5.066397818475386
1e-07
(2.0482701179415322e+21, 0.01)
[2.39479152 3.875

/Users/shilpakancharla/anaconda/lib/python2.7/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 6.60341881768e-17
  ' condition number: {}'.format(rcond), RuntimeWarning)
/Users/shilpakancharla/anaconda/lib/python2.7/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 3.3547508623e-18
  ' condition number: {}'.format(rcond), RuntimeWarning)
/Users/shilpakancharla/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
